<a href="https://colab.research.google.com/github/rawanaldaneen/pytorch_row/blob/main/neuralNetwork_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>